In [1]:
import pandas as pd
import sqlalchemy as sa
import sqlite3
import openai
import pyodbc
import pypyodbc as odbc

In [2]:
''''
server='like4.database.windows.net'
database='like'
connection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
conn=odbc.connect(connection_string)
'''

"'\nserver='like4.database.windows.net'\ndatabase='like'\nconnection_string='Driver={ODBC Driver 18 for SQL Server};Server=tcp:like4.database.windows.net,1433;Database=like;Uid=CloudSA42bee827;Pwd={XP2U@X3R5EOQ23};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'\nconn=odbc.connect(connection_string)\n"

In [5]:
''''
sql=''' 
#SELECT count(user_id)
#from dbo.Users
'''
cursor=conn.cursor()
cursor.execute(sql)
dataset=cursor.fetchall()
print(dataset)
'''

'\ncursor=conn.cursor()\ncursor.execute(sql)\ndataset=cursor.fetchall()\nprint(dataset)\n'

In [3]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= 'm.abuhassan%40likecard.com' # use %40 inplace of @
pass1='password' # use %40 inplace of @   

# Create a connection string for SQL Server using Windows authentication
connection_string = f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'

In [4]:
engine = sa.create_engine(connection_string, echo=True, connect_args={'autocommit': True}, fast_executemany=True)


In [25]:
sql_query = '''
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_NAME IN (SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE')
    and TABLE_NAME like 'dim_product' 
    or TABLE_NAME like 'fact_trans%' 
    
'''
df = pd.read_sql(sql_query, engine)

2024-01-04 15:16:34,533 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME], [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-04 15:16:34,534 INFO sqlalchemy.engine.Engine [cached since 581.1s ago] ("\nSELECT \n    TABLE_SCHEMA,\n    TABLE_NAME,\n    COLUMN_NAME,\n    DATA_TYPE\nFROM \n    INFORMATION_SCHEMA.COLUMNS\nWHERE \n    TABLE_NAME IN (SEL ... (16 characters truncated) ... ROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE')\n    and TABLE_NAME like 'dim_product' \n    or TABLE_NAME like 'fact_trans%' \n    \n", 'dbo')
2024-01-04 15:16:35,108 INFO sqlalchemy.engine.Engine 
SELECT 
    TABLE_SCHEMA,
    TABLE_NAME,
    COLUMN_NAME,
    DATA_TYPE
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_NAME IN (SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLE

In [26]:
table_schema=df.loc[0][0]

C:\Users\moham\AppData\Local\Temp\ipykernel_8176\1307101668.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  table_schema=df.loc[0][0]


In [27]:
df=df.drop(columns=['TABLE_SCHEMA'])


In [28]:
df.columns

Index(['TABLE_NAME', 'COLUMN_NAME', 'DATA_TYPE'], dtype='object')

In [29]:
df_dict = df.to_dict(orient='records')
df_dict

[{'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'OrderDateTime',
  'DATA_TYPE': 'datetime2'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'user_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'order_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'store_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'payment_method_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'bank_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'product_id',
  'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions', 'COLUMN_NAME': 'qty', 'DATA_TYPE': 'int'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'cost',
  'DATA_TYPE': 'float'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'coupon_code',
  'DATA_TYPE': 'varchar'},
 {'TABLE_NAME': 'fact_transactions',
  'COLUMN_NAME': 'fees',
  'DATA_TYPE': 'float'},
 {'TABL

In [30]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [46]:
text = 'what is product name that has the most purchases'

In [47]:
def generate_sql_query(table_names,text,table_schema):
    prompt = """You are a ChatGPT language model that can generate concise  SQL server queries. Please provide a natural language input text and I will generate the corresponding SQL query **without using descriptive words only sql query** in function calls or expressions for example replace DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) with DATE_SUB(CURRENT_DATE(),  1 ) . Use only function names and parentheses. the tables names , coulmns names and datatyps in this dict {}  and the tables schema is {} and Input: {} SQL Query (without descriptive words)  and note that you have multible tables so you can join the needed tables as needed:""".format(df_dict, table_schema, text)
    request = openai.ChatCompletion.create(
        engine="gpt-35-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )
    pythonCode = request.choices[0].message.content
    return pythonCode    
query = generate_sql_query(df_dict,text,table_schema)
query


'SELECT TOP 1 t2.product_name\nFROM dbo.fact_transactions t1\nJOIN dbo.dim_products t2 \n    ON t1.product_id = t2.id\nGROUP BY t2.product_name\nORDER BY SUM(t1.qty) DESC;'

In [48]:
print(query)

SELECT TOP 1 t2.product_name
FROM dbo.fact_transactions t1
JOIN dbo.dim_products t2 
    ON t1.product_id = t2.id
GROUP BY t2.product_name
ORDER BY SUM(t1.qty) DESC;


In [ ]:
def executeSQLquery(table_name,text,schema_list ,max_retries):
    retry_count = 0

    while retry_count < max_retries:
        try:
            query = generate_sql_query(table_name,text,schema_list)
            result = spark.sql(query)
            return result.collect()[0][0]
        except Exception as e:
            print(f"Error executing query: {e}")
            retry_count += 1
            if retry_count < max_retries:
                print(f"Retrying... (Attempt {retry_count}/{max_retries})")
            else:
                print("Max retry attempts reached. Exiting.")
                raise

In [32]:
sql_query = f"SELECT count(*) FROM {table_schema}.dim_allusers"
df = pd.read_sql(sql_query, engine)

2024-01-04 12:16:57,345 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME], [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-04 12:16:57,346 INFO sqlalchemy.engine.Engine [cached since 6001s ago] ('SELECT count(*) FROM dbo.dim_allusers', 'dbo')
2024-01-04 12:16:57,832 INFO sqlalchemy.engine.Engine SELECT count(*) FROM dbo.dim_allusers
2024-01-04 12:16:57,837 INFO sqlalchemy.engine.Engine [raw sql] ()


In [9]:
sql_query = '''
SELECT 
    table_name,
    table_schema
FROM 
    INFORMATION_SCHEMA.TABLES
WHERE 
    table_type = 'BASE TABLE'
ORDER BY 
    table_name ASC
'''
df = pd.read_sql(sql_query, engine)

2024-01-04 14:02:07,282 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2024-01-04 14:02:07,282 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-04 14:02:07,831 INFO sqlalchemy.engine.Engine SELECT schema_name()
2024-01-04 14:02:07,831 INFO sqlalchemy.engine.Engine [generated in 0.00103s] ()
2024-01-04 14:02:08,517 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2024-01-04 14:02:08,517 INFO sqlalchemy.engine.Engine [generated in 0.00230s] ()
2024-01-04 14:02:08,751 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME], [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2024-01-04 14:02:08,751 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ("\nSELECT \n    table_name,\n    table_schema\nFROM \n    INFORMAT